## Abdulkadir TAŞDELEN
### Bu dosyada önemli modellerin farklı batch size'lara göre k-fold CV sonucu elde edilen sonuçları içermektedir.

### Grid Search ile farklı parametrelerin testi


Last Update: 31.08.2024

Örnek Makaleler:
https://ms.hmb.gov.tr/uploads/2024/07/03-186_13-Turkiyede-Butce-Gelirlerinin-Tahmini.pdf
https://www.proquest.com/openview/b7712f88b52716855e3916cd47adeac6/1?cbl=396471&pq-origsite=gscholar&parentSessionId=VpA7%2F6IztQXgaL0%2BZm%2F074%2FnVBSVNxeyvFPQ%2BDFLaiQ%3D


## Data Processing

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, Normalizer

read_file=['v002_data_.csv',
            'previous_1.csv',
            'previous_1_zeros.csv',
            'previous_1_filtered.csv',
]
selected_file = read_file[1]


# Örnek veri kümesini oku
#data = pd.read_csv('v002_data_.csv', delimiter=";", index_col=0, skip_blank_lines=True)
#data = pd.read_csv('previous_1.csv', delimiter=";", index_col=0, skip_blank_lines=True)
#data = pd.read_csv('previous_1_zeros.csv', delimiter=";", index_col=0, skip_blank_lines=True)

data = pd.read_csv(selected_file, delimiter=";", index_col=0, skip_blank_lines=True)

print(data.isna().sum())  


# Sütun adlarını temizleme
data.columns = data.columns.str.strip()

# Virgülleri noktalarla değiştirme
for col in data.columns:
    if data[col].dtype == 'object':  # Eğer veri türü nesne (string) ise
        data[col] = data[col].str.replace(',', '.').astype(float, errors='ignore')
        
data = data.replace(' ', np.nan)

print(data['WSINS (KT)'])

# WSINS (KT) sütunundaki veriyi sayısal tipe dönüştürme
data['WSINS (KT)'] = data['WSINS (KT)'].str.replace(',', '.').astype(float, errors='ignore')

# Eksik olan veriyi doğrusal enterpolasyon ile doldurma
data['WSINS (KT)'] = data['WSINS (KT)'].interpolate(method='linear', inplace=False)

data = data.dropna()     


print(data.isna().sum())  
   
        
        
# Zaman damgalarını datetime formatına dönüştürme
data.index = pd.to_datetime(data.index, format='%d.%m.%Y %H:%M')


# İlk birkaç satırı görüntüle
print(f'Record size: {data.__len__()}')
print(data.columns.tolist())

# İlk birkaç satırı görüntüle
print(data.head())

print(data.columns)

# Boş değerleri kontrol etme
print(data.isnull().sum())  # Hangi sütunlarda eksik veri olduğunu gösterir

# Tarih/zaman sütunlarını sayısal verilere dönüştürme
data['HistoryfileUTCTime'] = pd.to_datetime(data.index)  # Eğer tarih/zaman verileri indeks olarak varsa
# Tarih/saat sütununu indeks olarak ayarla
data.set_index('HistoryfileUTCTime', inplace=True)

 

# Tarih/zaman sütununu X'ten çıkarın ve sadece sayısal verileri kullanın
#X = data.drop(['dB/km', 'HistoryfileUTCTime','1H RAIN(MM)','Refrence','RSL', 'dB'], axis=1)
#X = data.drop(['dB/km', 'Refrence','1H RAIN (MM)','WSINS (KT)','RSL', 'dB'], axis=1)
X = data.drop(['dB/km', 'Refrence','1H RAIN (MM)' ,'RSL', 'dB'], axis=1)
y = data['dB/km']


print(X.head()) 
print(X.count) 
print(X.isnull().sum())  # Hangi sütunlarda eksik veri olduğunu gösterir



# Veriyi numpy array'e çevirme (eğer DataFrame ise)
X = X.to_numpy() if isinstance(X, pd.DataFrame) else X
y = y.to_numpy() if isinstance(y, pd.DataFrame) else y



select_norm=['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'MaxAbsScaler', 'Normalizer']
selected_norm = select_norm[0]
# Veriyi normalize etme
if selected_norm == 'StandardScaler':
    # Skaler Normalizasyon
    scaler = StandardScaler()
elif selected_norm == 'MinMaxScaler':
    # MinMaxScaler Normalizasyon
    scaler = MinMaxScaler()
elif selected_norm == 'MaxAbsScale':
    # MaxAbsScaler ile veriyi normalize etme
    scaler = MaxAbsScaler()
elif selected_norm == 'Normalizerr':
    # Normalizer ile veriyi normalize etme
    scaler = Normalizer()
elif selected_norm == 'RobustScaler':
    # RobustScaler ile veriyi normalize etme
    scaler = RobustScaler()
    
    
    """
    StandardScaler: Veriyi ortalama 0 ve standart sapma 1 olacak şekilde normalize eder.
    MinMaxScaler: Veriyi belirli bir aralığa (genellikle 0 ile 1) dönüştürür.
    RobustScaler: Aykırı değerlere karşı dayanıklıdır, medyan ve IQR kullanır.
    MaxAbsScaler: Veriyi mutlak maksimum değere böler, genellikle negatif değerler için kullanılır.
    Normalizer: Her veri örneğini belirli bir norm ile böler, veri örneklerinin normunu 1 yapar.
    """

X = scaler.fit_transform(X)



MOR_1A              0
TEMP (ｰC)           0
RH (%)              0
PAINS (HPA)         0
1H RAIN (MM)        0
WSINS (KT)      17639
RSL                 0
Refrence            0
dB                  0
dB/km               0
dtype: int64
History file UTC Time
1.06.2014 21:00     5.64
1.06.2014 21:01     5.25
1.06.2014 21:02     4.47
1.06.2014 21:03     4.28
1.06.2014 21:04     5.25
                   ...  
2.07.2014 07:25    15.36
2.07.2014 07:26    20.22
2.07.2014 07:27    12.44
2.07.2014 07:28    22.16
2.07.2014 07:29    17.88
Name: WSINS (KT), Length: 39074, dtype: object
MOR_1A          0
TEMP (ｰC)       0
RH (%)          0
PAINS (HPA)     0
1H RAIN (MM)    0
WSINS (KT)      0
RSL             0
Refrence        0
dB              0
dB/km           0
dtype: int64
Record size: 39045
['MOR_1A', 'TEMP (ｰC)', 'RH (%)', 'PAINS (HPA)', '1H RAIN (MM)', 'WSINS (KT)', 'RSL', 'Refrence', 'dB', 'dB/km']
                      MOR_1A TEMP (ｰC) RH (%) PAINS (HPA) 1H RAIN (MM)  \
History file UTC Time   

# Hibrit Model Kodu (XGBoost + LSTM-GRU)


In [ ]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Flatten
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from itertools import product

# Define parameter ranges
"""n_estimators_list = [100, 200, 300]
learning_rate_list = [0.01, 0.1, 0.2]
max_depth_list = [6, 8, 10, 12]
subsample_list = [0.7, 0.8, 0.9]
colsample_bytree_list = [0.7, 0.8, 0.9]"""
n_estimators_list = [200]
learning_rate_list = [0.1]
max_depth_list = [10]
subsample_list = [0.8]
colsample_bytree_list = [0.8]

# Create parameter combinations
param_combinations = list(product(n_estimators_list, learning_rate_list, max_depth_list, subsample_list, colsample_bytree_list))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Define the batch size, unit sizes, and dropout sizes
batch_sizes = [64]
unit_sizes = [64]
dropout_sizes = [0.2]

model_n = 'Ensemble_XGBoost_LSTM_GRU'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
folder_main = f'models_exp/{model_n}'
os.makedirs(folder_main, exist_ok=True)

# Results list to store performance metrics
results = []

for batch_size in batch_sizes:
    for unit_size in unit_sizes:
        for dropout_size in dropout_sizes:
            for params in param_combinations:
                n_estimators, learning_rate, max_depth, subsample, colsample_bytree = params
                
                fold_no = 1
                all_fold_histories = []
                fold_results = []

                kf = KFold(n_splits=10, shuffle=True, random_state=42)

                for train_index, val_index in kf.split(X_train):
                    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

                    # XGBoost model
                    xgb_model = xgb.XGBRegressor(
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree
                    )
                    xgb_model.fit(X_train_fold, y_train_fold)
                    X_train_fold_transformed = xgb_model.predict(X_train_fold).reshape(-1, 1)
                    X_val_fold_transformed = xgb_model.predict(X_val_fold).reshape(-1, 1)
                    X_test_transformed = xgb_model.predict(X_test).reshape(-1, 1)

                    # Reshape the data for LSTM-GRU
                    X_train_fold_transformed = np.array(X_train_fold_transformed).reshape((X_train_fold_transformed.shape[0], X_train_fold_transformed.shape[1], 1))
                    X_val_fold_transformed = np.array(X_val_fold_transformed).reshape((X_val_fold_transformed.shape[0], X_val_fold_transformed.shape[1], 1))
                    X_test_transformed = np.array(X_test_transformed).reshape((X_test_transformed.shape[0], X_test_transformed.shape[1], 1))

                    # Define the LSTM-GRU model
                    model = Sequential()
                    model.add(LSTM(units=unit_size, activation='tanh', input_shape=(X_train_fold_transformed.shape[1], X_train_fold_transformed.shape[2]), return_sequences=True))
                    model.add(Dropout(dropout_size))
                    model.add(GRU(units=unit_size, activation='tanh', return_sequences=True))
                    model.add(Dropout(dropout_size))
                    model.add(Flatten())
                    model.add(Dense(100, activation='relu'))
                    model.add(Dropout(dropout_size))
                    model.add(Dense(1))  # Output layer for regression

                    # Compile the model
                    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae', 'mse'])

                    # Train the model
                    history = model.fit(X_train_fold_transformed, y_train_fold, validation_data=(X_val_fold_transformed, y_val_fold), epochs=100, batch_size=batch_size)

                    # Store fold history
                    all_fold_histories.append(history.history)

                    # Calculate RMSE for training, validation, and test sets
                    train_rmse = np.sqrt(history.history['mse'][-1])
                    val_rmse = np.sqrt(history.history['val_mse'][-1])

                    test_results = model.evaluate(X_test_transformed, y_test, verbose=0)
                    test_rmse = np.sqrt(test_results[2])

                    # Store fold results
                    fold_results.append({
                        'Fold': fold_no,
                        'Training Size': len(X_train_fold),
                        'Validation Size': len(X_val_fold),
                        'Test Size': len(X_test_transformed),
                        'Training Loss': history.history['loss'][-1],
                        'Validation Loss': history.history['val_loss'][-1],
                        'Training MAE': history.history['mae'][-1],
                        'Validation MAE': history.history['val_mae'][-1],
                        'Training MSE': history.history['mse'][-1],
                        'Validation MSE': history.history['val_mse'][-1],
                        'Test Loss': test_results[0],
                        'Test MAE': test_results[1],
                        'Test MSE': test_results[2],
                        'Training RMSE': train_rmse,
                        'Validation RMSE': val_rmse,
                        'Test RMSE': test_rmse,
                    })

                    fold_no += 1

                # Average fold histories
                average_history = {
                    'Epoch': np.arange(1, 101),
                    'Loss': np.mean(np.array([h['loss'] for h in all_fold_histories]).T, axis=1),
                    'Val_Loss': np.mean(np.array([h['val_loss'] for h in all_fold_histories]).T, axis=1),
                    'MAE': np.mean(np.array([h['mae'] for h in all_fold_histories]).T, axis=1),
                    'Val_MAE': np.mean(np.array([h['val_mae'] for h in all_fold_histories]).T, axis=1),
                    'MSE': np.mean(np.array([h['mse'] for h in all_fold_histories]).T, axis=1),
                    'Val_MSE': np.mean(np.array([h['val_mse'] for h in all_fold_histories]).T, axis=1),
                }

                # Average fold results
                average_fold_results = {
                    'Training Size': len(X_train_fold),
                    'Validation Size': len(X_val_fold),
                    'Test Size': len(X_test_transformed),
                    'Average Training Loss': np.mean([result['Training Loss'] for result in fold_results if not np.isnan(result['Training Loss'])]),
                    'Average Validation Loss': np.mean([result['Validation Loss'] for result in fold_results if not np.isnan(result['Validation Loss'])]),
                    'Average Test Loss': np.mean([result['Test Loss'] for result in fold_results if not np.isnan(result['Test Loss'])]),
                    'Average Training MAE': np.mean([result['Training MAE'] for result in fold_results if not np.isnan(result['Training MAE'])]),
                    'Average Validation MAE': np.mean([result['Validation MAE'] for result in fold_results if not np.isnan(result['Validation MAE'])]),
                    'Average Test MAE': np.mean([result['Test MAE'] for result in fold_results if not np.isnan(result['Test MAE'])]),
                    'Average Training MSE': np.mean([result['Training MSE'] for result in fold_results if not np.isnan(result['Training MSE'])]),
                    'Average Validation MSE': np.mean([result['Validation MSE'] for result in fold_results if not np.isnan(result['Validation MSE'])]),
                    'Average Test MSE': np.mean([result['Test MSE'] for result in fold_results if not np.isnan(result['Test MSE'])]),
                    'Average Training RMSE': np.mean([result['Training RMSE'] for result in fold_results if not np.isnan(result['Training RMSE'])]),
                    'Average Validation RMSE': np.mean([result['Validation RMSE'] for result in fold_results if not np.isnan(result['Validation RMSE'])]),
                    'Average Test RMSE': np.mean([result['Test RMSE'] for result in fold_results if not np.isnan(result['Test RMSE'])]),
                }

                # Store the results
                results.append({
                    'Params': params,
                    'Average Training Loss': average_fold_results['Average Training Loss'],
                    'Average Validation Loss': average_fold_results['Average Validation Loss'],
                    'Average Test Loss': average_fold_results['Average Test Loss'],
                    'Average Training MAE': average_fold_results['Average Training MAE'],
                    'Average Validation MAE': average_fold_results['Average Validation MAE'],
                    'Average Test MAE': average_fold_results['Average Test MAE'],
                    'Average Training MSE': average_fold_results['Average Training MSE'],
                    'Average Validation MSE': average_fold_results['Average Validation MSE'],
                    'Average Test MSE': average_fold_results['Average Test MSE'],
                    'Average Training RMSE': average_fold_results['Average Training RMSE'],
                    'Average Validation RMSE': average_fold_results['Average Validation RMSE'],
                    'Average Test RMSE': average_fold_results['Average Test RMSE'],
                })

                # Save results to Excel
                results_df = pd.DataFrame(results)
                results_df.to_excel(os.path.join(folder_main, f'{model_n}_{timestamp}_results.xlsx'), index=False)

print("Model eğitimi ve değerlendirmesi tamamlandı.")


In [5]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
from itertools import product
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Flatten
from keras.optimizers import Adam

"""n_estimators_list = [100, 200, 300]
learning_rate_list = [0.01, 0.1, 0.2]
max_depth_list = [6, 8, 10, 12]
subsample_list = [0.7, 0.8, 0.9]
colsample_bytree_list = [0.7, 0.8, 0.9]"""
n_estimators_list = [200]
learning_rate_list = [0.1]
max_depth_list = [12]
subsample_list = [0.8]
colsample_bytree_list = [0.8]

# Create parameter combinations
param_combinations = list(product(n_estimators_list, learning_rate_list, max_depth_list, subsample_list, colsample_bytree_list))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Define the batch size, unit sizes, and dropout sizes
batch_sizes = [64]
unit_sizes = [64]
dropout_sizes = [0.2]

model_n = 'Ensemble_XGBoost_LSTM_GRU'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
folder_main = f'models_exp/{model_n}'
os.makedirs(folder_main, exist_ok=True)

# Results list to store performance metrics
results = []

for batch_size in batch_sizes:
    for unit_size in unit_sizes:
        for dropout_size in dropout_sizes:
            for params in param_combinations:
                n_estimators, learning_rate, max_depth, subsample, colsample_bytree = params
                
                fold_no = 1
                all_fold_histories = []
                fold_results = []

                kf = KFold(n_splits=10, shuffle=True, random_state=42)

                for train_index, val_index in kf.split(X_train):
                    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

                    # XGBoost model
                    xgb_model = xgb.XGBRegressor(
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree
                    )
                    xgb_model.fit(X_train_fold, y_train_fold)
                    X_train_fold_transformed = xgb_model.predict(X_train_fold).reshape(-1, 1)
                    X_val_fold_transformed = xgb_model.predict(X_val_fold).reshape(-1, 1)
                    X_test_transformed = xgb_model.predict(X_test).reshape(-1, 1)

                    # Reshape the data
                    X_train_fold_transformed = np.array(X_train_fold_transformed).reshape((X_train_fold_transformed.shape[0], X_train_fold_transformed.shape[1], 1))
                    X_val_fold_transformed = np.array(X_val_fold_transformed).reshape((X_val_fold_transformed.shape[0], X_val_fold_transformed.shape[1], 1))
                    X_test_transformed = np.array(X_test_transformed).reshape((X_test_transformed.shape[0], X_test_transformed.shape[1], 1))

                    # Define the hybrid model
                    model = Sequential()
                    model.add(LSTM(units=unit_size, activation='tanh', input_shape=(X_train_fold_transformed.shape[1], X_train_fold_transformed.shape[2]), return_sequences=True))
                    model.add(Dropout(dropout_size))
                    model.add(GRU(units=unit_size, activation='tanh', return_sequences=True))
                    model.add(Dropout(dropout_size))
                    model.add(Flatten())
                    model.add(Dense(100, activation='relu'))
                    model.add(Dropout(dropout_size))
                    model.add(Dense(1))  # Output layer for regression

                    # Compile the model
                    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae', 'mse'])

                    # Train the model
                    history = model.fit(X_train_fold_transformed, y_train_fold, validation_data=(X_val_fold_transformed, y_val_fold), epochs=10, batch_size=batch_size)

                    # Store fold history
                    all_fold_histories.append(history.history)

                    # Calculate RMSE for training, validation, and test sets
                    train_rmse = np.sqrt(history.history['mse'][-1])
                    val_rmse = np.sqrt(history.history['val_mse'][-1])

                    test_results = model.evaluate(X_test_transformed, y_test, verbose=0)
                    test_rmse = np.sqrt(test_results[2])

                    # Store fold results
                    fold_results.append({
                        'Fold': fold_no,
                        'Training Size': len(X_train_fold_transformed),
                        'Validation Size': len(X_val_fold_transformed),
                        'Test Size': len(X_test_transformed),            
                        'Training Loss': history.history['loss'][-1],
                        'Validation Loss': history.history['val_loss'][-1],
                        'Training MAE': history.history['mae'][-1],
                        'Validation MAE': history.history['val_mae'][-1],
                        'Training MSE': history.history['mse'][-1],
                        'Validation MSE': history.history['val_mse'][-1],
                        'Test Loss': test_results[0],
                        'Test MAE': test_results[1],
                        'Test MSE': test_results[2],
                        'Training RMSE': train_rmse,
                        'Validation RMSE': val_rmse,
                        'Test RMSE': test_rmse,
                        'XGBoost Params': params
                    })

                    fold_no += 1

                # Tüm fold'ların sonuçlarını ortalama alma
                if all_fold_histories:
                    average_history = {
                        'Epoch': np.arange(1, 101),
                        'Loss': np.mean([h['loss'] for h in all_fold_histories], axis=0),
                        'Val_Loss': np.mean([h['val_loss'] for h in all_fold_histories], axis=0),
                        'MAE': np.mean([h['mae'] for h in all_fold_histories], axis=0),
                        'Val_MAE': np.mean([h['val_mae'] for h in all_fold_histories], axis=0),
                        'MSE': np.mean([h['mse'] for h in all_fold_histories], axis=0),
                        'Val_MSE': np.mean([h['val_mse'] for h in all_fold_histories], axis=0),
                    }

                    # Ortalama RMSE hesaplama
                    average_rmse = {
                        'Epoch': average_history['Epoch'],
                        'Train_RMSE': np.sqrt(average_history['MSE']),
                        'Val_RMSE': np.sqrt(average_history['Val_MSE'])
                    }

                    # Ortalama fold sonuçlarını hesapla
                    average_fold_results = {
                        'Training Size': len(X_train_fold_transformed),
                        'Validation Size': len(X_val_fold_transformed),
                        'Test Size': len(X_test_transformed),  
                        'Average Training Loss': np.mean([result['Training Loss'] for result in fold_results if not np.isnan(result['Training Loss'])]),
                        'Average Validation Loss': np.mean([result['Validation Loss'] for result in fold_results if not np.isnan(result['Validation Loss'])]),
                        'Average Test Loss': np.mean([result['Test Loss'] for result in fold_results if not np.isnan(result['Test Loss'])]),
                        'Average Training MAE': np.mean([result['Training MAE'] for result in fold_results if not np.isnan(result['Training MAE'])]),
                        'Average Validation MAE': np.mean([result['Validation MAE'] for result in fold_results if not np.isnan(result['Validation MAE'])]),
                        'Average Test MAE': np.mean([result['Test MAE'] for result in fold_results if not np.isnan(result['Test MAE'])]),
                        'Average Training MSE': np.mean([result['Training MSE'] for result in fold_results if not np.isnan(result['Training MSE'])]),
                        'Average Validation MSE': np.mean([result['Validation MSE'] for result in fold_results if not np.isnan(result['Validation MSE'])]),
                        'Average Test MSE': np.mean([result['Test MSE'] for result in fold_results if not np.isnan(result['Test MSE'])]),
                        'Average Training RMSE': np.mean([result['Training RMSE'] for result in fold_results if not np.isnan(result['Training RMSE'])]),
                        'Average Validation RMSE': np.mean([result['Validation RMSE'] for result in fold_results if not np.isnan(result['Validation RMSE'])]),
                        'Average Test RMSE': np.mean([result['Test RMSE'] for result in fold_results if not np.isnan(result['Test RMSE'])]),
                        'XGBoost Params': params
                    }

                    # Sonuçları sakla
                    results.append(average_fold_results)

                    # Performans grafiklerini kaydet
                    plt.figure(figsize=(12, 8))

                    # Grafik çizimi sırasında uzunluk kontrolü ekleyin
                    if len(average_rmse['Epoch']) == len(average_rmse['Train_RMSE']) and len(average_rmse['Epoch']) == len(average_rmse['Val_RMSE']):
                        plt.plot(average_rmse['Epoch'], average_rmse['Train_RMSE'], label='Training RMSE')
                        plt.plot(average_rmse['Epoch'], average_rmse['Val_RMSE'], label='Validation RMSE')
                    else:
                        print("Warning: Length mismatch in RMSE data.")
                        print(f"Epoch length: {len(average_rmse['Epoch'])}")
                        print(f"Train RMSE length: {len(average_rmse['Train_RMSE'])}")
                        print(f"Val RMSE length: {len(average_rmse['Val_RMSE'])}")

                    plt.title(f'{model_n} - RMSE over epochs')
                    plt.xlabel('Epoch')
                    plt.ylabel('RMSE')
                    plt.legend()
                    plt.savefig(f'{folder_main}/rmse_plot_{params[0]}_{params[1]}_{params[2]}_{params[3]}_{params[4]}.png')
                    plt.close()

# Sonuçları DataFrame'e dönüştür ve kaydet
results_df = pd.DataFrame(results)
results_df.to_csv(f'{folder_main}/xgb_lstm_gru_results.csv', index=False)


C:\Users\abdul\AppData\Local\Temp\ipykernel_33640\28345785.py:59: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/10
495/495 [==============================] - 4s 3ms/step - loss: 0.0198 - mae: 0.0863 - mse: 0.0198 - val_loss: 0.0042 - val_mae: 0.0305 - val_mse: 0.0042
Epoch 2/10
495/495 [==============================] - 1s 3ms/step - loss: 0.0036 - mae: 0.0446 - mse: 0.0036 - val_loss: 0.0044 - val_mae: 0.0326 - val_mse: 0.0044
Epoch 3/10
495/495 [==============================] - 1s 3ms/step - loss: 0.0028 - mae: 0.0392 - mse: 0.0028 - val_loss: 0.0042 - val_mae: 0.0297 - val_mse: 0.0042
Epoch 4/10
495/495 [==============================] - 1s 3ms/step - loss: 0.0024 - mae: 0.0367 - mse: 0.0024 - val_loss: 0.0044 - val_mae: 0.0335 - val_mse: 0.0044
Epoch 5/10
495/495 [==============================] - 1s 3ms/step - loss: 0.0023 - mae: 0.0355 - mse: 0.0023 - val_loss: 0.0043 - val_mae: 0.0304 - val_mse: 0.0043
Epoch 6/10
495/495 [==============================] - 1s 3ms/step - loss: 0.0020 - mae: 0.0334 - mse: 0.0020 - val_loss: 0.0044 - val_mae: 0.0343 - val_mse: 0.0044
Epoch 7/10
495/4

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Epoch length: 100
Train RMSE length: 10
Val RMSE length: 10


PermissionError: [Errno 13] Permission denied: 'models_exp/Ensemble_XGBoost_LSTM_GRU/xgb_lstm_gru_results.csv'

In [6]:
results_df = pd.DataFrame(results)
results_df.to_csv(f'{folder_main}/xgb_lstm_gru_results.csv', index=False)

In [3]:
# Sonuçları DataFrame'e dönüştür ve kaydet
results_df = pd.DataFrame(results)
results_df.to_csv(f'{folder_main}/xgb_lstm_gru_results.csv', index=False)

In [8]:
print(results_df)

   Training Size  Validation Size  Test Size  Average Training Loss  \
0          31626             3514       3905               0.001434   

   Average Validation Loss  Average Test Loss  Average Training MAE  \
0                 0.005547           0.005105              0.026449   

   Average Validation MAE  Average Test MAE  Average Training MSE  \
0                0.032305          0.031817              0.001434   

   Average Validation MSE  Average Test MSE  Average Training RMSE  \
0                0.005547          0.005105               0.037858   

   Average Validation RMSE  Average Test RMSE            XGBoost Params  
0                 0.074104            0.07139  (200, 0.1, 12, 0.8, 0.8)  


# Data collector from separate excel files

In [15]:
import os
import pandas as pd
from datetime import datetime

# Klasör yolu ve hedef dosya yolu
#folder_path = "grid_analiz"
folder_path = "models_exp/test_results"
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f"merged_output_{timestamp}.xlsx"
output_sheet_name = "Merged Results"

# Tüm Excel dosyalarının ikinci satırlarını toplamak için bir liste
merged_data = []

# Klasördeki tüm Excel dosyalarını tarama
for file_namex in os.listdir(folder_path):
    print(f"Processing file: {file_namex}")
    
    if file_namex.endswith(".xlsx") or file_namex.endswith(".xls"):
        file_path = os.path.join(folder_path, file_namex)
        file_path = file_path.replace("\\", "/")
        
        # Excel dosyasını oku ve her bir sheet'i tarayarak veri bul
        try:
            # Dosyadaki tüm sheet'lerin isimlerini al
            xls = pd.ExcelFile(file_path)
            sheet_found = False
            
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                print(f"Sheet: {sheet_name} - Satır sayısı: {len(df)}")

                # Eğer sheet'te en az iki satır varsa ikinci satırı al
                """print(len(df))
                print(df)"""
                if len(df) >= 1:
                    if not merged_data: 
                        # İlk sütuna dosya adını eklemek için sütun başlıklarını güncelle
                        columns = df.columns.tolist()
                        merged_data.append(columns)
                    
                    # 2. satırı al ve başına dosya adını ekle
                    second_row = df.iloc[0].tolist()
                    merged_data.append(second_row)
                    sheet_found = True
                    break  # Veriyi bulduktan sonra diğer sheet'leri arama

            if not sheet_found:
                print(f"{file_namex} dosyasında yeterli veri yok, atlanıyor.")
        
        except Exception as e:
            print(f"Error reading {file_namex}: {e}")

# Eğer `merged_data` boşsa, işlem yapmadan çıkış yap
if not merged_data:
    print("No data found. Please check the folder path or sheet names.")
else:
    # Tüm verileri bir DataFrame'e dönüştür
    merged_df = pd.DataFrame(merged_data[1:], columns=merged_data[0])

    # Sonuçları tek bir Excel dosyasına yaz
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        merged_df.to_excel(writer, sheet_name=output_sheet_name, index=False)

    print(f"İkinci satırlar {output_file} dosyasına kaydedildi.")


Processing file: exp_16_128_0.2_20240904_224552_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_128_0.3_20240904_230527_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_128_0.4_20240904_232541_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_128_0.5_20240904_234523_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_32_0.2_20240904_193857_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_32_0.3_20240904_195304_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_32_0.4_20240904_200719_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_32_0.5_20240904_202137_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_64_0.2_20240904_203638_combined.xlsx
Sheet: Cumulative Results - Satır sayısı: 1
Processing file: exp_16_64_0.3_20240904_205136_comb

# Hibrit Model FİNAL v01 (XGBoost + LSTM-GRU) bitti

In [11]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split
import xgboost as xgb
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Flatten
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Batch size'leri tanımla
batch_sizes = [48]
unit_sizes = [128]
dropout_sizes = [0.2]



"""n_estimators_list = [100, 200, 300]
learning_rate_list = [0.01, 0.1, 0.2]
max_depth_list = [6, 8, 10, 12]
subsample_list = [0.7, 0.8, 0.9]
colsample_bytree_list = [0.7, 0.8, 0.9]"""
n_estimators = 300
learning_rate = 0.1
max_depth = 10
subsample = 0.8
colsample_bytree = 0.8

                   

model_n = 'Ensemble_XGBoost_LSTM_GRU'
# Klasör oluşturma
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
folder_main = f'models_exp/{model_n}'
os.makedirs(folder_main, exist_ok=True)

folder = f'{folder_main}/exp_{timestamp}'
os.makedirs(folder, exist_ok=True)

for batch_size in batch_sizes:
    for unit_size in unit_sizes:
        for dropout_size in dropout_sizes:
            fold_no = 1
            all_fold_histories = []
            params = []
            fold_results = []

            # Model parameters
            params.append({
                'File name': selected_file,
                'Total Fold': 10,
                'LSTM Units': unit_size,
                'GRU Units': unit_size,
                'Dropout Rate': dropout_size,
                'Dense Units': 100,
                'Batch Size': batch_size,
                'Epochs': 100,
                'LearningRate': '0.001',
                'Normalizasyon': selected_norm,
            })

            kf = KFold(n_splits=params[0]['Total Fold'], shuffle=False)

            for train_index, val_index in kf.split(X_train):
                # Create train/validation splits within each fold
                X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

                # Feature extraction with XGBoost
                # XGBoost model
                xgb_model = xgb.XGBRegressor(
                    n_estimators=n_estimators,
                    learning_rate=learning_rate,
                    max_depth=max_depth,
                    subsample=subsample,
                    colsample_bytree=colsample_bytree
                )
                xgb_model.fit(X_train_fold, y_train_fold)
                X_train_fold_xgb = xgb_model.predict(X_train_fold).reshape(-1, 1)
                X_val_fold_xgb = xgb_model.predict(X_val_fold).reshape(-1, 1)
                X_test_xgb = xgb_model.predict(X_test).reshape(-1, 1)

                # Reshape the data
                X_train_fold_xgb = np.array(X_train_fold_xgb).reshape((X_train_fold_xgb.shape[0], X_train_fold_xgb.shape[1], 1))
                X_val_fold_xgb = np.array(X_val_fold_xgb).reshape((X_val_fold_xgb.shape[0], X_val_fold_xgb.shape[1], 1))
                X_test_xgb = np.array(X_test_xgb).reshape((X_test_xgb.shape[0], X_test_xgb.shape[1], 1))

                # Define the model
                model = Sequential()
                model.add(LSTM(units=params[0]['LSTM Units'], activation='tanh', input_shape=(X_train_fold_xgb.shape[1], X_train_fold_xgb.shape[2]), return_sequences=True))
                model.add(Dropout(params[0]['Dropout Rate']))
                model.add(GRU(units=params[0]['GRU Units'], activation='tanh', return_sequences=True))
                model.add(Dropout(params[0]['Dropout Rate']))
                model.add(Flatten())
                model.add(Dense(params[0]['Dense Units'], activation='relu'))
                model.add(Dropout(params[0]['Dropout Rate']))
                model.add(Dense(1))  # Output layer for regression

                # Compile the model
                model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae', 'mse'])

                # Train the model
                history = model.fit(X_train_fold_xgb, y_train_fold, validation_data=(X_val_fold_xgb, y_val_fold), epochs=params[0]['Epochs'], batch_size=params[0]['Batch Size'])

                # Store fold history
                all_fold_histories.append(history.history)

                # Calculate RMSE for training, validation, and test sets
                train_rmse = np.sqrt(history.history['mse'][-1])
                val_rmse = np.sqrt(history.history['val_mse'][-1])

                test_results = model.evaluate(X_test_xgb, y_test, verbose=0)
                test_rmse = np.sqrt(test_results[2])

                # Calculate R^2 scores
                y_pred_train = model.predict(X_train_fold_xgb).flatten()
                y_pred_val = model.predict(X_val_fold_xgb).flatten()
                y_pred_test = model.predict(X_test_xgb).flatten()

                r2_train = r2_score(y_train_fold, y_pred_train)
                r2_val = r2_score(y_val_fold, y_pred_val)
                r2_test = r2_score(y_test, y_pred_test)

                # Store fold results
                fold_results.append({
                    'Fold': fold_no,
                    'Training Size': len(X_train_fold),
                    'Validation Size': len(X_val_fold),
                    'Test Size': len(X_test_xgb),
                    'Training Loss': history.history['loss'][-1],
                    'Validation Loss': history.history['val_loss'][-1],
                    'Training MAE': history.history['mae'][-1],
                    'Validation MAE': history.history['val_mae'][-1],
                    'Training MSE': history.history['mse'][-1],
                    'Validation MSE': history.history['val_mse'][-1],
                    'Test Loss': test_results[0],
                    'Test MAE': test_results[1],
                    'Test MSE': test_results[2],
                    'Training RMSE': train_rmse,
                    'Validation RMSE': val_rmse,
                    'Test RMSE': test_rmse,
                    'Training R^2': r2_train,
                    'Validation R^2': r2_val,
                    'Test R^2': r2_test
                })

                fold_no += 1

            # Tüm fold'ların sonuçlarını ortalama alma
            average_history = {
                'Epoch': np.arange(1, params[0]['Epochs'] + 1),
                'Loss': np.mean(np.array([h['loss'] for h in all_fold_histories]).T, axis=1),
                'Val_Loss': np.mean(np.array([h['val_loss'] for h in all_fold_histories]).T, axis=1),
                'MAE': np.mean(np.array([h['mae'] for h in all_fold_histories]).T, axis=1),
                'Val_MAE': np.mean(np.array([h['val_mae'] for h in all_fold_histories]).T, axis=1),
                'MSE': np.mean(np.array([h['mse'] for h in all_fold_histories]).T, axis=1),
                'Val_MSE': np.mean(np.array([h['val_mse'] for h in all_fold_histories]).T, axis=1),
            }

            # Ortalama RMSE hesaplama
            average_rmse = {
                'Epoch': average_history['Epoch'],
                'Train_RMSE': np.sqrt(average_history['MSE']),
                'Val_RMSE': np.sqrt(average_history['Val_MSE'])
            }

            # Ortalama fold sonuçlarını hesapla
            average_fold_results = {
                'Training Size': len(X_train_fold),
                'Validation Size': len(X_val_fold),
                'Test Size': len(X_test_xgb),
                'Average Training Loss': np.mean([result['Training Loss'] for result in fold_results if not np.isnan(result['Training Loss'])]),
                'Average Validation Loss': np.mean([result['Validation Loss'] for result in fold_results if not np.isnan(result['Validation Loss'])]),
                'Average Test Loss': np.mean([result['Test Loss'] for result in fold_results if not np.isnan(result['Test Loss'])]),
                'Average Training MAE': np.mean([result['Training MAE'] for result in fold_results if not np.isnan(result['Training MAE'])]),
                'Average Validation MAE': np.mean([result['Validation MAE'] for result in fold_results if not np.isnan(result['Validation MAE'])]),
                'Average Test MAE': np.mean([result['Test MAE'] for result in fold_results if not np.isnan(result['Test MAE'])]),
                'Average Training MSE': np.mean([result['Training MSE'] for result in fold_results if not np.isnan(result['Training MSE'])]),
                'Average Validation MSE': np.mean([result['Validation MSE'] for result in fold_results if not np.isnan(result['Validation MSE'])]),
                'Average Test MSE': np.mean([result['Test MSE'] for result in fold_results if not np.isnan(result['Test MSE'])]),
                'Average Training RMSE': np.mean([result['Training RMSE'] for result in fold_results if not np.isnan(result['Training RMSE'])]),
                'Average Validation RMSE': np.mean([result['Validation RMSE'] for result in fold_results if not np.isnan(result['Validation RMSE'])]),
                'Average Test RMSE': np.mean([result['Test RMSE'] for result in fold_results if not np.isnan(result['Test RMSE'])]),
                'Average Training R^2': np.mean([result['Training R^2'] for result in fold_results if not np.isnan(result['Training R^2'])]),
                'Average Validation R^2': np.mean([result['Validation R^2'] for result in fold_results if not np.isnan(result['Validation R^2'])]),
                'Average Test R^2': np.mean([result['Test R^2'] for result in fold_results if not np.isnan(result['Test R^2'])]),
            }

            # Sonuçları ve model parametrelerini Excel dosyasına kaydetme
            results_df = pd.DataFrame(average_history)
            params_df = pd.DataFrame(params)
            fold_results_df = pd.DataFrame(fold_results)
            average_rmse_df = pd.DataFrame(average_rmse)
            average_fold_results_df = pd.DataFrame([average_fold_results])

            # Klasör oluşturma
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            name_detailed = f"exp_{batch_size}_{unit_size}_{dropout_size}"
            folder_name = f'{folder}/{name_detailed}'
            os.makedirs(folder_name, exist_ok=True)

            # Dosya adı oluşturma
            file_name = f'{name_detailed}_{timestamp}.xlsx'  
            file_path = os.path.join(folder_name, file_name)
            file_path = file_path.replace("\\", "/")

            # Ortalama eğitim ve doğrulama kaybını çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_history['Epoch'], average_history['Loss'], label='Training Loss (average)')
            plt.plot(average_history['Epoch'], average_history['Val_Loss'], label='Validation Loss (average)')
            plt.title('Model Loss - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_loss.png'))
            plt.close()

            # Ortalama eğitim ve doğrulama MAE'yi çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_history['Epoch'], average_history['MAE'], label='Training MAE (average)')
            plt.plot(average_history['Epoch'], average_history['Val_MAE'], label='Validation MAE (average)')
            plt.title('Model MAE - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('MAE')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_mae.png'))
            plt.close()

            # Ortalama eğitim ve doğrulama MSE'yi çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_history['Epoch'], average_history['MSE'], label='Training MSE (average)')
            plt.plot(average_history['Epoch'], average_history['Val_MSE'], label='Validation MSE (average)')
            plt.title('Model MSE - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('MSE')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_mse.png'))
            plt.close()

            # Ortalama eğitim ve doğrulama RMSE'sini çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_rmse['Epoch'], average_rmse['Train_RMSE'], label='Training RMSE (average)')
            plt.plot(average_rmse['Epoch'], average_rmse['Val_RMSE'], label='Validation RMSE (average)')
            plt.title('Model RMSE - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('RMSE')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_rmse.png'))
            plt.close()
            
            # Excel dosyasına veri yazma
            with pd.ExcelWriter(file_path) as writer:
                results_df.to_excel(writer, sheet_name='Average History', index=False)
                params_df.to_excel(writer, sheet_name='Model Parameters', index=False)
                fold_results_df.to_excel(writer, sheet_name='Fold Results', index=False)
                average_rmse_df.to_excel(writer, sheet_name='Average RMSE', index=False)
                average_fold_results_df.to_excel(writer, sheet_name='Average Fold Results', index=False)

            print(f'Results saved to {file_path}')
            
            # İki veri çerçevesini yatay olarak birleştir (axis=1 ile)
            combined_df = pd.concat([params_df, average_fold_results_df], axis=1)

            file_name2 = f'{name_detailed}_{timestamp}_combined.xlsx'  
            file_path2 = os.path.join(folder, file_name2)
            file_path2 = file_path2.replace("\\", "/")
            
            # ExcelWriter ile dosyayı aç ve veri çerçevesini yaz
            with pd.ExcelWriter(file_path2, engine='openpyxl') as writer:
                combined_df.to_excel(writer, sheet_name='Cumulative Results', index=False)

            print(f"Results for cycle written to {file_path2}")

print("Model eğitimi ve değerlendirmesi tamamlandı.")


C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\4221429961.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
659/659 [==============================] - 4s 4ms/step - loss: 0.0670 - mae: 0.1687 - mse: 0.0670 - val_loss: 0.0070 - val_mae: 0.0577 - val_mse: 0.0070
Epoch 2/100
659/659 [==============================] - 2s 3ms/step - loss: 0.0067 - mae: 0.0635 - mse: 0.0067 - val_loss: 0.0035 - val_mae: 0.0349 - val_mse: 0.0035
Epoch 3/100
659/659 [==============================] - 2s 3ms/step - loss: 0.0052 - mae: 0.0551 - mse: 0.0052 - val_loss: 0.0032 - val_mae: 0.0299 - val_mse: 0.0032
Epoch 4/100
659/659 [==============================] - 2s 3ms/step - loss: 0.0047 - mae: 0.0526 - mse: 0.0047 - val_loss: 0.0031 - val_mae: 0.0295 - val_mse: 0.0031
Epoch 5/100
659/659 [==============================] - 2s 3ms/step - loss: 0.0043 - mae: 0.0500 - mse: 0.0043 - val_loss: 0.0032 - val_mae: 0.0300 - val_mse: 0.0032
Epoch 6/100
659/659 [==============================] - 2s 3ms/step - loss: 0.0040 - mae: 0.0479 - mse: 0.0040 - val_loss: 0.0032 - val_mae: 0.0301 - val_mse: 0.0032
Epoch 7/10

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\4221429961.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
659/659 [==============================] - 6s 5ms/step - loss: 0.0576 - mae: 0.1551 - mse: 0.0576 - val_loss: 0.0078 - val_mae: 0.0572 - val_mse: 0.0078
Epoch 2/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0062 - mae: 0.0605 - mse: 0.0062 - val_loss: 0.0043 - val_mae: 0.0346 - val_mse: 0.0043
Epoch 3/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0048 - mae: 0.0531 - mse: 0.0048 - val_loss: 0.0042 - val_mae: 0.0331 - val_mse: 0.0042
Epoch 4/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0043 - mae: 0.0503 - mse: 0.0043 - val_loss: 0.0041 - val_mae: 0.0315 - val_mse: 0.0041
Epoch 5/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0039 - mae: 0.0474 - mse: 0.0039 - val_loss: 0.0041 - val_mae: 0.0314 - val_mse: 0.0041
Epoch 6/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0036 - mae: 0.0455 - mse: 0.0036 - val_loss: 0.0041 - val_mae: 0.0312 - val_mse: 0.0041
Epoch 7/10

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\4221429961.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


...

Epoch 1/100
659/659 [==============================] - 6s 5ms/step - loss: 0.0613 - mae: 0.1611 - mse: 0.0613 - val_loss: 0.0102 - val_mae: 0.0604 - val_mse: 0.0102
Epoch 2/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0067 - mae: 0.0632 - mse: 0.0067 - val_loss: 0.0083 - val_mae: 0.0377 - val_mse: 0.0083
Epoch 3/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0052 - mae: 0.0547 - mse: 0.0052 - val_loss: 0.0082 - val_mae: 0.0367 - val_mse: 0.0082
Epoch 4/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0046 - mae: 0.0517 - mse: 0.0046 - val_loss: 0.0080 - val_mae: 0.0325 - val_mse: 0.0080
Epoch 5/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0042 - mae: 0.0491 - mse: 0.0042 - val_loss: 0.0081 - val_mae: 0.0379 - val_mse: 0.0081
Epoch 6/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0039 - mae: 0.0471 - mse: 0.0039 - val_loss: 0.0078 - val_mae: 0.0333 - val_mse: 0.0078
Epoch 7/10

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\4221429961.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
659/659 [==============================] - 6s 5ms/step - loss: 0.0616 - mae: 0.1599 - mse: 0.0616 - val_loss: 0.0077 - val_mae: 0.0554 - val_mse: 0.0077
Epoch 2/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0062 - mae: 0.0609 - mse: 0.0062 - val_loss: 0.0046 - val_mae: 0.0324 - val_mse: 0.0046
Epoch 3/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0049 - mae: 0.0536 - mse: 0.0049 - val_loss: 0.0047 - val_mae: 0.0346 - val_mse: 0.0047
Epoch 4/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0044 - mae: 0.0507 - mse: 0.0044 - val_loss: 0.0045 - val_mae: 0.0316 - val_mse: 0.0045
Epoch 5/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0041 - mae: 0.0485 - mse: 0.0041 - val_loss: 0.0045 - val_mae: 0.0307 - val_mse: 0.0045
Epoch 6/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0037 - mae: 0.0462 - mse: 0.0037 - val_loss: 0.0045 - val_mae: 0.0311 - val_mse: 0.0045
Epoch 7/10

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\4221429961.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
659/659 [==============================] - 6s 5ms/step - loss: 0.0673 - mae: 0.1701 - mse: 0.0673 - val_loss: 0.0089 - val_mae: 0.0605 - val_mse: 0.0089
Epoch 2/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0070 - mae: 0.0649 - mse: 0.0070 - val_loss: 0.0055 - val_mae: 0.0328 - val_mse: 0.0055
Epoch 3/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0053 - mae: 0.0554 - mse: 0.0053 - val_loss: 0.0054 - val_mae: 0.0324 - val_mse: 0.0054
Epoch 4/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0047 - mae: 0.0521 - mse: 0.0047 - val_loss: 0.0055 - val_mae: 0.0345 - val_mse: 0.0055
Epoch 5/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0043 - mae: 0.0499 - mse: 0.0043 - val_loss: 0.0053 - val_mae: 0.0314 - val_mse: 0.0053
Epoch 6/100
659/659 [==============================] - 3s 5ms/step - loss: 0.0040 - mae: 0.0480 - mse: 0.0040 - val_loss: 0.0054 - val_mae: 0.0324 - val_mse: 0.0054
Epoch 7/10

In [ ]:
model.save("trained_hyrid_model.h5")

# Hibrit Model FİNAL v02 TimeSeriesSplit (XGBoost + LSTM-GRU)

In [10]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit

import xgboost as xgb
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Flatten
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Batch size'leri tanımla
batch_sizes = [48]
unit_sizes = [128]
dropout_sizes = [0.2]

"""# Batch size'leri tanımla
batch_sizes = [64]
unit_sizes = [32, 64, 96, 128]
dropout_sizes = [0.2, 0.3, 0.4, 0.5]"""

"""n_estimators_list = [100, 200, 300]
learning_rate_list = [0.01, 0.1, 0.2]
max_depth_list = [6, 8, 10, 12]
subsample_list = [0.7, 0.8, 0.9]
colsample_bytree_list = [0.7, 0.8, 0.9]"""
n_estimators = 100
learning_rate = 0.1
max_depth = 10
subsample = 0.8
colsample_bytree = 0.8

                   

model_n = 'Ensemble_XGBoost_LSTM_GRU'
# Klasör oluşturma
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
folder_main = f'models_exp/{model_n}'
os.makedirs(folder_main, exist_ok=True)

folder = f'{folder_main}/exp_{timestamp}'
os.makedirs(folder, exist_ok=True)

for batch_size in batch_sizes:
    for unit_size in unit_sizes:
        for dropout_size in dropout_sizes:
            fold_no = 1
            all_fold_histories = []
            params = []
            fold_results = []

            # Model parameters
            params.append({
                'File name': selected_file,
                'Total Fold': 10,
                'LSTM Units': unit_size,
                'GRU Units': unit_size,
                'Dropout Rate': dropout_size,
                'Dense Units': 100,
                'Batch Size': batch_size,
                'Epochs': 100,
                'LearningRate': '0.001',
                'Normalizasyon': selected_norm,
            })

            """kf = KFold(n_splits=params[0]['Total Fold'], shuffle=False)

            for train_index, val_index in kf.split(X_train):
                # Create train/validation splits within each fold
                X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]"""
                
            # KFold yerine TimeSeriesSplit kullanımı
            tscv = TimeSeriesSplit(n_splits=params[0]['Total Fold'])

            for train_index, val_index in tscv.split(X_train):
                # Create train/validation splits within each fold
                X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
                
                # Feature extraction with XGBoost
                # XGBoost model
                xgb_model = xgb.XGBRegressor(
                    n_estimators=n_estimators,
                    learning_rate=learning_rate,
                    max_depth=max_depth,
                    subsample=subsample,
                    colsample_bytree=colsample_bytree
                )
                xgb_model.fit(X_train_fold, y_train_fold)
                X_train_fold_xgb = xgb_model.predict(X_train_fold).reshape(-1, 1)
                X_val_fold_xgb = xgb_model.predict(X_val_fold).reshape(-1, 1)
                X_test_xgb = xgb_model.predict(X_test).reshape(-1, 1)

                # Reshape the data
                X_train_fold_xgb = np.array(X_train_fold_xgb).reshape((X_train_fold_xgb.shape[0], X_train_fold_xgb.shape[1], 1))
                X_val_fold_xgb = np.array(X_val_fold_xgb).reshape((X_val_fold_xgb.shape[0], X_val_fold_xgb.shape[1], 1))
                X_test_xgb = np.array(X_test_xgb).reshape((X_test_xgb.shape[0], X_test_xgb.shape[1], 1))

                # Define the model
                model = Sequential()
                model.add(LSTM(units=params[0]['LSTM Units'], activation='tanh', input_shape=(X_train_fold_xgb.shape[1], X_train_fold_xgb.shape[2]), return_sequences=True))
                model.add(Dropout(params[0]['Dropout Rate']))
                model.add(GRU(units=params[0]['GRU Units'], activation='tanh', return_sequences=True))
                model.add(Dropout(params[0]['Dropout Rate']))
                model.add(Flatten())
                model.add(Dense(params[0]['Dense Units'], activation='relu'))
                model.add(Dropout(params[0]['Dropout Rate']))
                model.add(Dense(1))  # Output layer for regression

                # Compile the model
                model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae', 'mse'])

                # Train the model
                history = model.fit(X_train_fold_xgb, y_train_fold, validation_data=(X_val_fold_xgb, y_val_fold), epochs=params[0]['Epochs'], batch_size=params[0]['Batch Size'])

                # Store fold history
                all_fold_histories.append(history.history)

                # Calculate RMSE for training, validation, and test sets
                train_rmse = np.sqrt(history.history['mse'][-1])
                val_rmse = np.sqrt(history.history['val_mse'][-1])

                test_results = model.evaluate(X_test_xgb, y_test, verbose=0)
                test_rmse = np.sqrt(test_results[2])

                # Calculate R^2 scores
                y_pred_train = model.predict(X_train_fold_xgb).flatten()
                y_pred_val = model.predict(X_val_fold_xgb).flatten()
                y_pred_test = model.predict(X_test_xgb).flatten()

                r2_train = r2_score(y_train_fold, y_pred_train)
                r2_val = r2_score(y_val_fold, y_pred_val)
                r2_test = r2_score(y_test, y_pred_test)

                # Store fold results
                fold_results.append({
                    'Fold': fold_no,
                    'Training Size': len(X_train_fold),
                    'Validation Size': len(X_val_fold),
                    'Test Size': len(X_test_xgb),
                    'Training Loss': history.history['loss'][-1],
                    'Validation Loss': history.history['val_loss'][-1],
                    'Training MAE': history.history['mae'][-1],
                    'Validation MAE': history.history['val_mae'][-1],
                    'Training MSE': history.history['mse'][-1],
                    'Validation MSE': history.history['val_mse'][-1],
                    'Test Loss': test_results[0],
                    'Test MAE': test_results[1],
                    'Test MSE': test_results[2],
                    'Training RMSE': train_rmse,
                    'Validation RMSE': val_rmse,
                    'Test RMSE': test_rmse,
                    'Training R^2': r2_train,
                    'Validation R^2': r2_val,
                    'Test R^2': r2_test
                })

                fold_no += 1

            # Tüm fold'ların sonuçlarını ortalama alma
            average_history = {
                'Epoch': np.arange(1, params[0]['Epochs'] + 1),
                'Loss': np.mean(np.array([h['loss'] for h in all_fold_histories]).T, axis=1),
                'Val_Loss': np.mean(np.array([h['val_loss'] for h in all_fold_histories]).T, axis=1),
                'MAE': np.mean(np.array([h['mae'] for h in all_fold_histories]).T, axis=1),
                'Val_MAE': np.mean(np.array([h['val_mae'] for h in all_fold_histories]).T, axis=1),
                'MSE': np.mean(np.array([h['mse'] for h in all_fold_histories]).T, axis=1),
                'Val_MSE': np.mean(np.array([h['val_mse'] for h in all_fold_histories]).T, axis=1),
            }

            # Ortalama RMSE hesaplama
            average_rmse = {
                'Epoch': average_history['Epoch'],
                'Train_RMSE': np.sqrt(average_history['MSE']),
                'Val_RMSE': np.sqrt(average_history['Val_MSE'])
            }

            # Ortalama fold sonuçlarını hesapla
            average_fold_results = {
                'Training Size': len(X_train_fold),
                'Validation Size': len(X_val_fold),
                'Test Size': len(X_test_xgb),
                'Average Training Loss': np.mean([result['Training Loss'] for result in fold_results if not np.isnan(result['Training Loss'])]),
                'Average Validation Loss': np.mean([result['Validation Loss'] for result in fold_results if not np.isnan(result['Validation Loss'])]),
                'Average Test Loss': np.mean([result['Test Loss'] for result in fold_results if not np.isnan(result['Test Loss'])]),
                'Average Training MAE': np.mean([result['Training MAE'] for result in fold_results if not np.isnan(result['Training MAE'])]),
                'Average Validation MAE': np.mean([result['Validation MAE'] for result in fold_results if not np.isnan(result['Validation MAE'])]),
                'Average Test MAE': np.mean([result['Test MAE'] for result in fold_results if not np.isnan(result['Test MAE'])]),
                'Average Training MSE': np.mean([result['Training MSE'] for result in fold_results if not np.isnan(result['Training MSE'])]),
                'Average Validation MSE': np.mean([result['Validation MSE'] for result in fold_results if not np.isnan(result['Validation MSE'])]),
                'Average Test MSE': np.mean([result['Test MSE'] for result in fold_results if not np.isnan(result['Test MSE'])]),
                'Average Training RMSE': np.mean([result['Training RMSE'] for result in fold_results if not np.isnan(result['Training RMSE'])]),
                'Average Validation RMSE': np.mean([result['Validation RMSE'] for result in fold_results if not np.isnan(result['Validation RMSE'])]),
                'Average Test RMSE': np.mean([result['Test RMSE'] for result in fold_results if not np.isnan(result['Test RMSE'])]),
                'Average Training R^2': np.mean([result['Training R^2'] for result in fold_results if not np.isnan(result['Training R^2'])]),
                'Average Validation R^2': np.mean([result['Validation R^2'] for result in fold_results if not np.isnan(result['Validation R^2'])]),
                'Average Test R^2': np.mean([result['Test R^2'] for result in fold_results if not np.isnan(result['Test R^2'])]),
            }

            # Sonuçları ve model parametrelerini Excel dosyasına kaydetme
            results_df = pd.DataFrame(average_history)
            params_df = pd.DataFrame(params)
            fold_results_df = pd.DataFrame(fold_results)
            average_rmse_df = pd.DataFrame(average_rmse)
            average_fold_results_df = pd.DataFrame([average_fold_results])

            # Klasör oluşturma
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            name_detailed = f"exp_{batch_size}_{unit_size}_{dropout_size}"
            folder_name = f'{folder}/{name_detailed}'
            os.makedirs(folder_name, exist_ok=True)

            # Dosya adı oluşturma
            file_name = f'{name_detailed}_{timestamp}.xlsx'  
            file_path = os.path.join(folder_name, file_name)
            file_path = file_path.replace("\\", "/")

            # Ortalama eğitim ve doğrulama kaybını çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_history['Epoch'], average_history['Loss'], label='Training Loss (average)')
            plt.plot(average_history['Epoch'], average_history['Val_Loss'], label='Validation Loss (average)')
            plt.title('Model Loss - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_loss.png'))
            plt.close()

            # Ortalama eğitim ve doğrulama MAE'yi çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_history['Epoch'], average_history['MAE'], label='Training MAE (average)')
            plt.plot(average_history['Epoch'], average_history['Val_MAE'], label='Validation MAE (average)')
            plt.title('Model MAE - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('MAE')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_mae.png'))
            plt.close()

            # Ortalama eğitim ve doğrulama MSE'yi çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_history['Epoch'], average_history['MSE'], label='Training MSE (average)')
            plt.plot(average_history['Epoch'], average_history['Val_MSE'], label='Validation MSE (average)')
            plt.title('Model MSE - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('MSE')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_mse.png'))
            plt.close()

            # Ortalama eğitim ve doğrulama RMSE'sini çizme
            plt.figure(figsize=(12, 6))
            plt.plot(average_rmse['Epoch'], average_rmse['Train_RMSE'], label='Training RMSE (average)')
            plt.plot(average_rmse['Epoch'], average_rmse['Val_RMSE'], label='Validation RMSE (average)')
            plt.title('Model RMSE - Average across folds')
            plt.xlabel('Epochs')
            plt.ylabel('RMSE')
            plt.legend()
            plt.savefig(os.path.join(folder_name, 'average_rmse.png'))
            plt.close()
            
            # Excel dosyasına veri yazma
            with pd.ExcelWriter(file_path) as writer:
                results_df.to_excel(writer, sheet_name='Average History', index=False)
                params_df.to_excel(writer, sheet_name='Model Parameters', index=False)
                fold_results_df.to_excel(writer, sheet_name='Fold Results', index=False)
                average_rmse_df.to_excel(writer, sheet_name='Average RMSE', index=False)
                average_fold_results_df.to_excel(writer, sheet_name='Average Fold Results', index=False)

            print(f'Results saved to {file_path}')
            
            # İki veri çerçevesini yatay olarak birleştir (axis=1 ile)
            combined_df = pd.concat([params_df, average_fold_results_df], axis=1)

            file_name2 = f'{name_detailed}_{timestamp}_combined.xlsx'  
            file_path2 = os.path.join(folder, file_name2)
            file_path2 = file_path2.replace("\\", "/")
            
            # ExcelWriter ile dosyayı aç ve veri çerçevesini yaz
            with pd.ExcelWriter(file_path2, engine='openpyxl') as writer:
                combined_df.to_excel(writer, sheet_name='Cumulative Results', index=False)

            print(f"Results for cycle written to {file_path2}")

print("Model eğitimi ve değerlendirmesi tamamlandı.")



C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\2027109278.py:84: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
67/67 [==============================] - 3s 10ms/step - loss: 0.0805 - mae: 0.2015 - mse: 0.0805 - val_loss: 0.0237 - val_mae: 0.1013 - val_mse: 0.0237
Epoch 2/100
67/67 [==============================] - 0s 5ms/step - loss: 0.0118 - mae: 0.0830 - mse: 0.0118 - val_loss: 0.0171 - val_mae: 0.0708 - val_mse: 0.0171
Epoch 3/100
67/67 [==============================] - 0s 4ms/step - loss: 0.0066 - mae: 0.0630 - mse: 0.0066 - val_loss: 0.0165 - val_mae: 0.0636 - val_mse: 0.0165
Epoch 4/100
67/67 [==============================] - 0s 4ms/step - loss: 0.0053 - mae: 0.0565 - mse: 0.0053 - val_loss: 0.0164 - val_mae: 0.0625 - val_mse: 0.0164
Epoch 5/100
67/67 [==============================] - 0s 4ms/step - loss: 0.0048 - mae: 0.0544 - mse: 0.0048 - val_loss: 0.0162 - val_mae: 0.0604 - val_mse: 0.0162
Epoch 6/100
67/67 [==============================] - 0s 4ms/step - loss: 0.0045 - mae: 0.0521 - mse: 0.0045 - val_loss: 0.0163 - val_mae: 0.0606 - val_mse: 0.0163
Epoch 7/100
67/67 [==

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\2027109278.py:84: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
134/134 [==============================] - 3s 7ms/step - loss: 0.0479 - mae: 0.1433 - mse: 0.0479 - val_loss: 0.0145 - val_mae: 0.0591 - val_mse: 0.0145
Epoch 2/100
134/134 [==============================] - 0s 4ms/step - loss: 0.0063 - mae: 0.0614 - mse: 0.0063 - val_loss: 0.0128 - val_mae: 0.0524 - val_mse: 0.0128
Epoch 3/100
134/134 [==============================] - 1s 4ms/step - loss: 0.0053 - mae: 0.0558 - mse: 0.0053 - val_loss: 0.0133 - val_mae: 0.0545 - val_mse: 0.0133
Epoch 4/100
134/134 [==============================] - 1s 4ms/step - loss: 0.0049 - mae: 0.0541 - mse: 0.0049 - val_loss: 0.0132 - val_mae: 0.0576 - val_mse: 0.0132
Epoch 5/100
134/134 [==============================] - 1s 4ms/step - loss: 0.0043 - mae: 0.0505 - mse: 0.0043 - val_loss: 0.0130 - val_mae: 0.0524 - val_mse: 0.0130
Epoch 6/100
134/134 [==============================] - 1s 4ms/step - loss: 0.0040 - mae: 0.0486 - mse: 0.0040 - val_loss: 0.0133 - val_mae: 0.0529 - val_mse: 0.0133
Epoch 7/10

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\2027109278.py:84: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


...

Epoch 1/100
533/533 [==============================] - 5s 5ms/step - loss: 0.0175 - mae: 0.0827 - mse: 0.0175 - val_loss: 0.0054 - val_mae: 0.0409 - val_mse: 0.0054
Epoch 2/100
533/533 [==============================] - 2s 3ms/step - loss: 0.0048 - mae: 0.0527 - mse: 0.0048 - val_loss: 0.0052 - val_mae: 0.0395 - val_mse: 0.0052
Epoch 3/100
533/533 [==============================] - 2s 3ms/step - loss: 0.0042 - mae: 0.0490 - mse: 0.0042 - val_loss: 0.0056 - val_mae: 0.0433 - val_mse: 0.0056
Epoch 4/100
533/533 [==============================] - 2s 3ms/step - loss: 0.0038 - mae: 0.0461 - mse: 0.0038 - val_loss: 0.0053 - val_mae: 0.0411 - val_mse: 0.0053
Epoch 5/100
533/533 [==============================] - 2s 3ms/step - loss: 0.0034 - mae: 0.0434 - mse: 0.0034 - val_loss: 0.0052 - val_mae: 0.0401 - val_mse: 0.0052
Epoch 6/100
533/533 [==============================] - 2s 3ms/step - loss: 0.0031 - mae: 0.0415 - mse: 0.0031 - val_loss: 0.0054 - val_mae: 0.0419 - val_mse: 0.0054
Epoch 7/10

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\2027109278.py:84: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
599/599 [==============================] - 4s 4ms/step - loss: 0.0140 - mae: 0.0744 - mse: 0.0140 - val_loss: 0.0060 - val_mae: 0.0396 - val_mse: 0.0060
Epoch 2/100
599/599 [==============================] - 2s 4ms/step - loss: 0.0042 - mae: 0.0492 - mse: 0.0042 - val_loss: 0.0060 - val_mae: 0.0389 - val_mse: 0.0060
Epoch 3/100
599/599 [==============================] - 2s 4ms/step - loss: 0.0037 - mae: 0.0455 - mse: 0.0037 - val_loss: 0.0065 - val_mae: 0.0445 - val_mse: 0.0065
Epoch 4/100
599/599 [==============================] - 2s 4ms/step - loss: 0.0034 - mae: 0.0435 - mse: 0.0034 - val_loss: 0.0060 - val_mae: 0.0389 - val_mse: 0.0060
Epoch 5/100
599/599 [==============================] - 2s 4ms/step - loss: 0.0031 - mae: 0.0416 - mse: 0.0031 - val_loss: 0.0066 - val_mae: 0.0460 - val_mse: 0.0066
Epoch 6/100
599/599 [==============================] - 2s 4ms/step - loss: 0.0030 - mae: 0.0402 - mse: 0.0030 - val_loss: 0.0061 - val_mae: 0.0403 - val_mse: 0.0061
Epoch 7/10

C:\Users\abdul\AppData\Local\Temp\ipykernel_13120\2027109278.py:84: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]


Epoch 1/100
666/666 [==============================] - 6s 5ms/step - loss: 0.0153 - mae: 0.0769 - mse: 0.0153 - val_loss: 0.0058 - val_mae: 0.0377 - val_mse: 0.0058
Epoch 2/100
666/666 [==============================] - 3s 5ms/step - loss: 0.0046 - mae: 0.0514 - mse: 0.0046 - val_loss: 0.0059 - val_mae: 0.0386 - val_mse: 0.0059
Epoch 3/100
666/666 [==============================] - 3s 5ms/step - loss: 0.0039 - mae: 0.0472 - mse: 0.0039 - val_loss: 0.0059 - val_mae: 0.0388 - val_mse: 0.0059
Epoch 4/100
666/666 [==============================] - 3s 5ms/step - loss: 0.0035 - mae: 0.0440 - mse: 0.0035 - val_loss: 0.0059 - val_mae: 0.0397 - val_mse: 0.0059
Epoch 5/100
666/666 [==============================] - 3s 5ms/step - loss: 0.0032 - mae: 0.0418 - mse: 0.0032 - val_loss: 0.0058 - val_mae: 0.0379 - val_mse: 0.0058
Epoch 6/100
666/666 [==============================] - 3s 5ms/step - loss: 0.0029 - mae: 0.0392 - mse: 0.0029 - val_loss: 0.0059 - val_mae: 0.0396 - val_mse: 0.0059
Epoch 7/10